# r/place Analysis 

## by: Brogan Pietrocini & Kaden Buckley

## Unzip .csv's and convert to single parquet process

In [14]:
#To create csvs
import gzip
import pandas as pd
import os

def read_gzip_csv(file_path, output_path):
    try:
        with gzip.open(file_path, 'rt', encoding='utf-8') as file:
            df = pd.read_csv(file)
        df.to_csv(output_path, index=False)
        return True
    except Exception as e:
        print(f"Error processing file {file_path}: {e}")
        return False

def process_all_gzip_in_folder(input_folder, output_folder):
    if not os.path.exists(input_folder):
        print(f"Input folder {input_folder} does not exist.")
        return 
    
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    processed_files = 0
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.gzip'):
            input_file_path = os.path.join(input_folder, file_name)
            output_file_path = os.path.join(output_folder, file_name[:-5] + '.csv')  # Removes '.gz'

            if read_gzip_csv(input_file_path, output_file_path):
                processed_files += 1
                print(f"Processed {file_name}")

    print(f"Total files processed: {processed_files}")

# Set your input and output folder paths
input_folder_path = r'C:\Users\broga\OneDrive\Desktop\r_place_zipped'
output_folder_path = r'C:\Users\broga\OneDrive\Desktop\r_place_unzipped'

process_all_gzip_in_folder(input_folder_path, output_folder_path)

Processed 2023_place_canvas_history-000000000000.csv.gzip
Processed 2023_place_canvas_history-000000000001.csv.gzip
Processed 2023_place_canvas_history-000000000002.csv.gzip
Processed 2023_place_canvas_history-000000000003.csv.gzip
Processed 2023_place_canvas_history-000000000004.csv.gzip
Processed 2023_place_canvas_history-000000000005.csv.gzip
Processed 2023_place_canvas_history-000000000006.csv.gzip
Processed 2023_place_canvas_history-000000000007.csv.gzip
Processed 2023_place_canvas_history-000000000008.csv.gzip
Processed 2023_place_canvas_history-000000000009.csv.gzip
Processed 2023_place_canvas_history-000000000010.csv.gzip
Processed 2023_place_canvas_history-000000000011.csv.gzip
Processed 2023_place_canvas_history-000000000012.csv.gzip
Processed 2023_place_canvas_history-000000000013.csv.gzip
Processed 2023_place_canvas_history-000000000014.csv.gzip
Processed 2023_place_canvas_history-000000000015.csv.gzip
Processed 2023_place_canvas_history-000000000016.csv.gzip
Processed 2023

In [6]:
# Combining to single parquet
import os
import pyarrow.csv as pv
import pyarrow.parquet as pq

def csv_folder_to_parquet(csv_folder_path, output_parquet_file):
    # Initialize a Parquet writer
    parquet_writer = None

    for csv_file in os.listdir(csv_folder_path):
        if csv_file.endswith('.csv'):
            csv_file_path = os.path.join(csv_folder_path, csv_file)
            
            # Read the CSV file into a PyArrow Table
            table = pv.read_csv(csv_file_path)
            
            # Write to a Parquet file in a streaming manner
            if parquet_writer is None:
                parquet_writer = pq.ParquetWriter(output_parquet_file, table.schema)
            parquet_writer.write_table(table)

    # Close the Parquet writer
    if parquet_writer:
        parquet_writer.close()


csv_folder_path = r"C:\Users\broga\OneDrive\Desktop\r_place_unzipped"  
output_parquet_file = r"C:\Users\broga\OneDrive\Desktop\r_place_parquet\combined.parquet"
csv_folder_to_parquet(csv_folder_path, output_parquet_file)


In [7]:
#Sanity Check
import pyarrow.parquet as pq

parquet_file_path = r"C:\Users\broga\OneDrive\Desktop\r_place_parquet\combined.parquet"
parquet_file = pq.ParquetFile(parquet_file_path)

print(f"Number of entries (rows) in the r/place Parquet file: {parquet_file.metadata.num_rows:,d}")

Number of entries (rows) in the Parquet file: 132,224,375


# Summary Statistics

In [ ]:
import polars as pl

# Define the path to your Parquet file
parquet_file_path = r"C:\Users\broga\OneDrive\Desktop\r_place_parquet\combined.parquet"

# Create a lazy dataframe from the Parquet file
lazy_df = pl.scan_parquet(parquet_file_path)


## Number of Unique Users

In [9]:
#Number of unique users

# Count the number of unique users
unique_users_count = (
    lazy_df.select(pl.col("user").n_unique())
    .collect()  # Triggers the actual computation
)

print("Number of unique users:", unique_users_count)


Number of unique users: shape: (1, 1)
┌─────────┐
│ user    │
│ ---     │
│ u32     │
╞═════════╡
│ 8589732 │
└─────────┘


# Top 5 Pixel Colors by Count

In [21]:
#Top 5 Pixel Colors by Count

# Count the occurrences of each 'pixel_color', sort them, and get the top 5
top_colors = (
    lazy_df.group_by("pixel_color")
           .agg(pl.len().alias("count"))
           .sort(by="count",descending=True)  # Sort in descending order
           .limit(5)  # Limit to top 5
           .collect()  # Trigger the actual computation
)

print("Top 5 pixel colors by count:")
print(top_colors)


Top 5 pixel colors by count:
shape: (5, 2)
┌─────────────┬──────────┐
│ pixel_color ┆ count    │
│ ---         ┆ ---      │
│ str         ┆ u32      │
╞═════════════╪══════════╡
│ #000000     ┆ 29041166 │
│ #FFFFFF     ┆ 27893555 │
│ #FF4500     ┆ 18201679 │
│ #FFD635     ┆ 9445982  │
│ #3690EA     ┆ 7595884  │
└─────────────┴──────────┘


# Top Pixels by User

In [25]:
#Top pixels by user

# Group by 'user', count the number of rows (pixels) for each user, and sort the results
user_pixel_counts = (
    lazy_df
    .groupby('user')
    .agg([
        pl.count('user').alias('pixel_count')
    ])
    .sort(by='pixel_count', descending=True)
)

# Collect the results (this is where the actual computation happens)
sorted_user_pixel_counts_collected = user_pixel_counts.collect()

# Display the result
print(sorted_user_pixel_counts_collected)

C:\Users\broga\AppData\Local\Temp\ipykernel_3940\3134637935.py:6: DeprecationWarning: `groupby` is deprecated. It has been renamed to `group_by`.
  .groupby('user')


shape: (8_589_732, 2)
┌───────────────────────────────────┬─────────────┐
│ user                              ┆ pixel_count │
│ ---                               ┆ ---         │
│ str                               ┆ u32         │
╞═══════════════════════════════════╪═════════════╡
│ kN91vAk/cntIE4qOgH6b06ZWInymLuAe… ┆ 7504        │
│ oCgWHk6Cfsa52vAOJBPqBDyKN+OH3Obx… ┆ 6285        │
│ /YNNvIErSA5atGXTER7fymCxIY/9xvun… ┆ 5331        │
│ bRwFe1qJhazPc7ur8S6hK8U0J0iPFVap… ┆ 4510        │
│ ay5U7RmL4QAj/p06ODdMbVh7lc1LqvfZ… ┆ 4390        │
│ …                                 ┆ …           │
│ yYbQeOkMqLCI0TvoV2PEW9VDomYW1bag… ┆ 1           │
│ LAT7BiU3xKtFObypSPxS7mie6KmWniy4… ┆ 1           │
│ Y6Vy8lzTmRTaRxdC+LotnRrBDWtoLAAE… ┆ 1           │
│ z4Zr5aHfTk2zRNcTZv16lCqWI5QofmMm… ┆ 1           │
│ fq+MLSJpnPLaGGhylWY/vuW0I7m7RP1i… ┆ 1           │
└───────────────────────────────────┴─────────────┘


## Amount of "users" over 1,440 Pixels Placed

In [1]:
import polars as pl

# Define the path to your Parquet file
parquet_file_path = r"C:\Users\broga\OneDrive\Desktop\r_place_parquet\combined.parquet"

# Create a lazy dataframe from the Parquet file
lazy_df = pl.scan_parquet(parquet_file_path)

# Calculate the pixel count for each user, then filter and count those with pixel_count greater than 1440
user_pixel_count_over_1440 = (
    lazy_df.group_by("user")
           .agg(pl.len().alias("pixel_count"))
           .filter(pl.col("pixel_count") > 1440)
           .collect()  
           .shape[0]  
)

print("Number of users with pixel_count greater than 1440:", user_pixel_count_over_1440)


Number of users with pixel_count greater than 1440: 67
